<a href="https://colab.research.google.com/github/Tsatthienmach/AiSummerCode/blob/master/week4/traveling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random

n = 5    # so luong thanh pho
m = 400  # so luong ca the trong quan the
n_generations = 1000 # so vong doi
losses = []          # de ve bieu de losses


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/Tsatthienmach/AiSummerCode.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 63 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), done.
/content/cloned-repo
_config.yml  documentations  README.md	week2  week4


In [4]:

# load data
def load_data():
    map = []
    file = open('/content/cloned-repo/week4/data_route.txt', 'r')
    lines = file.readlines()
    
    # bo header
    for i in range(1, len(lines)):
        strings = lines[i].split(',')
        
        # bo cot dau tien
        prices = [int(s.strip()) for s in strings[1:]]
        map.append(prices)
    file.close()
    return map

map = load_data()
print('load_data')
print(map)



load_data
[[1000, 12, 1000, 5, 7], [12, 1000, 14, 1000, 18], [1000, 14, 1000, 6, 19], [5, 1000, 6, 1000, 2], [7, 18, 19, 2, 1000]]


In [0]:

# tao individual
def create_individual():
    # list cac thanh pho    
    return [random.randint(1,n) for _ in range(n)]

# tinh loss
def compute_loss(individual):
    i = 0
    price = 0
    while i < n-1:
        a = individual[i] - 1
        b = individual[i+1] - 1
        price += map[a][b]
        i += 1
    # cong voi quang duong tp cuoi ve tp dau
    start = individual[0] - 1
    finish = individual[n-1] - 1
    price += map[finish][start]
    
    # Kiem tra xem individual co chua het thanh pho khong
    s = set(individual)
    
    price += ((n-len(s)) * 1000)
    # print(s , price)
    return price


# tinh fitness
def compute_fitness(individual):
    loss = compute_loss(individual)
    return 1 / (1 + loss)

# chon loc
def selection(sorted_population):
    index1 = random.randint(0, m-1)
    while True:
        index2 = random.randint(0, m-1)
        if index2 != index1:
            break
    individual = sorted_population[index1]
    if index2 > index1:
        individual = sorted_population[index2]
    return individual

# lai ghep:
def crossover(individual1, individual2, crossover_rate = 0.9):
    individual1_new = individual1.copy()
    individual2_new = individual2.copy()
    
    for i in range(n):
        if random.random() < crossover_rate:
            individual1_new[i] = individual2[i]
            individual2_new[i] = individual1[i]            
    
    return individual1_new, individual2_new

# dot bien
def mutate(individual, mutation_rate = 0.05):
    individual_m = individual.copy()
    
    for i in range(n):
        if random.random() < mutation_rate:
            individual_m[i] = random.randint(1,n)
        
    return individual_m

# tao quan the moi
def create_new_population(sorted_old_population):
    # luu vao losses
    losses.append(compute_loss(sorted_old_population[-1]))
    
    # in cac gia tri tot nhat qua tung doi
    # print(losses[-1])
    new_population = []
    while len(new_population) < m-2:
        # chon loc
        individual1 = selection(sorted_old_population)
        individual2 = selection(sorted_old_population)

        # lai ghep
        individual_c1, individual_c2 = crossover(individual1, individual2)

        # dot bien
        individual_m1 = mutate(individual_c1)
        individual_m2 = mutate(individual_c2)
    
        # cho vao quan the moi
        new_population.append(individual_m1)
        new_population.append(individual_m2)
    
    # cho 2 con dep nhat cua quan the cu vao quan the moi
    new_population.append(sorted_old_population[-1])
    new_population.append(sorted_old_population[-2])

    return new_population


In [6]:

# tao quan the ban dau
population = [create_individual() for _ in range(m)]

for _ in range(n_generations):
    sorted_old_population = sorted(population, key = compute_fitness)
    population = create_new_population(sorted_old_population)
    
# hien thi tuyen duong ngan nhat
route_min = sorted_old_population[-1]
route_min.append(sorted_old_population[-1][0])
print('duong di ngan nhat: ', route_min, 'chi phi: ', losses[-1])

duong di ngan nhat:  [4, 5, 1, 2, 3, 4] chi phi:  41
